In [1]:
import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from itertools import product
import warnings
from fredapi import Fred
from dotenv import load_dotenv
import os

warnings.filterwarnings('ignore')


def arima_grid_search(data, p_range, d_range, q_range):
    best_aic = np.inf  # Set a large value for best AIC to start
    best_params = None  # Placeholder for the best parameters
    
    # Loop through all possible combinations of (p, d, q)
    for p, d, q in product(p_range, d_range, q_range):
        try:
            # Create and fit ARIMA model for each parameter combination
            model = ARIMA(data, order=(p, d, q))
            results = model.fit()
            aic = results.aic  # Get the AIC value
            
            # If the AIC is lower than our best recorded AIC, update the best params
            if aic < best_aic:
                best_aic = aic
                best_params = (p, d, q)
            
            # Print progress
            print(f'ARIMA{(p,d,q)} - AIC:{aic}')
        except:
            # If the model fails to fit, we skip to the next combination
            continue
    
    print(f'Best ARIMA{best_params} - AIC:{best_aic}')
    return best_params


if __name__ == "__main__":
    # Fetch data
    # Load API_key from .env
    load_dotenv()
    API_KEY = os.getenv('API_KEY')

    # Initialize Fred API with your API key
    fred = Fred(api_key=API_KEY)

    data = fred.get_series('DGS10')
    data = pd.DataFrame(data, columns=['Interest Rate'])
    data.index.name = 'Date'
    data = data[(data.index >= '2010-01-01') & (data.index < '2020-01-01')]
    data.dropna(inplace=True)

    # Define the ranges for (p, d, q)
    p_range = range(0, 1)  # AR order
    d_range = range(0, 1 )  # Differencing
    q_range = range(0, 1)  # MA order

    # Perform ARIMA grid search to find the best parameters
    best_params = arima_grid_search(data, p_range, d_range, q_range)
    
    print("Best parameters (p, d, q):", best_params)


ARIMA(0, 0, 0) - AIC:4151.7745296624
Best ARIMA(0, 0, 0) - AIC:4151.7745296624
Best parameters (p, d, q): (0, 0, 0)
